# Team ByteMe notebook for OBL Data Innovation Challenge

   In this notebook we have described and implemented methods through which we were able to generate our regression models for the prediction task. We have used a combination of supervised ML models and Neural networks to compare the results. Stacked Average modelling was also done for predictions.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split as tts
import numpy as np
from sklearn.metrics import mean_squared_error as mse
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Utility functions

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def plot_vs_y(col):
    fig, ax = plt.subplots()
    ax.scatter(x = df[col], y = df['Price'])
    plt.ylabel('Price', fontsize=13)
    plt.xlabel(col, fontsize=13)
    plt.show()

def normal_dist(dataset, col, transform='None'):
    df = dataset.copy()
    if transform == 'log':
        df[col] = np.log(df[col])
    elif transform == 'boxcox':
        df[col], fitted_lambda = stats.boxcox(df['Price'])
        print('Fitted lambda for boxcox:', fitted_lambda)
    
    sns.distplot(df[col], fit=norm);
    (mu, sigma) = norm.fit(df[col])
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
    plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
                loc='best')
    plt.ylabel('Frequency')
    plt.title('SalePrice distribution')
    fig = plt.figure()
    res = stats.probplot(df[col], plot=plt)
    plt.show()
    
def train_model(model, x_train, y_train, x_val):
    model.fit(x_train, y_train)
    return model.predict(x_val).astype(int), model

def predict_test(model, filename, nn=False):
    x_test = pd.read_csv('test_obl.csv').drop('id', axis=1)
    x_test = extract_features(x_test)
    x_test.columns = x_train.columns
    if nn==True:
        pred = model.predict(x_test.values).astype(int)
    else:
        pred = model.predict(x_test).astype(int)
    create_submission(pred, filename)
    
def create_submission(pred, filename):
    sample = pd.read_csv('sample.csv')
    sample['Price'] = pred
    sample.to_csv(filename, index=False)
    print('Created file:', filename)

In [ ]:
df = pd.read_csv('training_obl.csv').drop('id', axis=1)
print(df.columns)

In [ ]:
corr = df.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

From the heatmap it can be observed that the `Nbedrooms, NWashrooms, ANB, Grade` are the only features that have a positive correlation with the `Price` of the houses.

# Distribution of the data

In [ ]:
normal_dist(df, 'Area(total)')
normal_dist(df, 'Area(total)', 'log')
normal_dist(df, 'Area(total)', 'boxcox')

The `Area(total)` does not follow a normal distribution and hence a form of transformation of the data will be required. The different areas are totalled and combined into a single column called `Area`

In [ ]:
df['Area(total)'] = df['Area(total)'] + df['Roof(Area)'] + df['Lawn(Area)']
normal_dist(df, 'Area(total)')
normal_dist(df, 'Area(total)', 'log')
normal_dist(df, 'Area(total)', 'boxcox')

Here, we can see that the `Area` column now follows a normal distribution and hence we can proceed with this transformation.

In [ ]:
normal_dist(df, 'Price')
normal_dist(df, 'Price', 'log')
normal_dist(df, 'Price', 'boxcox')

**Due to the bimodal nature of the distribution of `Price` column, we worked with transformation using *log* and *boxcox*.**

**However the results obtained did not better the ones obtained using non transformed features and was hence omitted from this notebook.**

In [ ]:
plt.hist(df['API'], bins=7)
api_counts = pd.DataFrame(df['API'].value_counts())
print('Unique values of API:', df['API'].nunique())

`API` can be divided into 7 bins based on the distribution and the numerical attribute can be converted into a discrete categorical attribute.

In [ ]:
api_counts.sort_index(inplace=False)
pd.cut(df['API'], 7)

The features selected are as shown below: 
1. `API` feature was dropped as it did not provide correlation with the `Price`.
2. `Nbedrooms` was squared to provide for higher weightage
3. `Rooms` feature was created by taking sum of the features `Nbedrooms` and `Nwashrooms`
4. `Floors` was converted into categorical

In [ ]:
X = df.iloc[:, :-1]
y = df['Price']

def extract_features(X):
    X['Nbedrooms'] = X['Nbedrooms']**2
    X['Rooms'] = X['Nbedrooms'] + X['Nwashrooms']
    floors = pd.get_dummies(X['Nfloors'])
    floors.columns = ['floors_'+str(i) for i in range(floors.shape[1])]
    
    #anb = pd.get_dummies(X['ANB'])
    #anb.columns = ['anb_'+str(i) for i in range(anb.shape[1])]
    
    #api = pd.get_dummies(X['API'])
    #api = pd.get_dummies(pd.cut(X['API'], 7, labels=['api_'+str(i) for i in range(7)]))
    #X = pd.concat([X, floors, api], axis=1)
    X = pd.concat([X, floors], axis=1)
    X.drop(['Roof(Area)','Lawn(Area)','Nfloors','API'], axis=1, inplace=True)
    #X.drop(['Roof(Area)','Lawn(Area)','Nfloors','API','Grade','ANB'], axis=1, inplace=True)
    return X

X = extract_features(X.copy())
#y_log = np.log(y)
x_train, x_val, y_train, y_val = tts(X, y, test_size=0.2, random_state=0)

In [ ]:
x_train.head()

## Supervised Models

1. Lasso Regression
2. ElasticNet
3. Kernel Ridge
4. Decision Tree
5. Random Forest
6. Gradient Boosting
7. XGBoost
8. Light Gradient Boosting

In [ ]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train.values)
    rmse = np.sqrt(-cross_val_score(model, x_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return rmse

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
tree = DecisionTreeRegressor(random_state=0)
score = rmsle_cv(tree)
print("Decision Tree score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
forest = RandomForestRegressor(n_estimators=100, random_state=0)
score = rmsle_cv(forest)
print("Random Forest(100) score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf=6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

### Models used for predictions

In [ ]:
XGBoost = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=10000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state=7, nthread = -1)
xgb_pred, xgb_model = train_model(XGBoost, x_train, y_train, x_val)
print('XGBoost RMSE:', rmse(xgb_pred, y_val))
predict_test(xgb_model, 'xgb-pred.csv')

In [ ]:
LGBoost = lgb.LGBMRegressor(objective='regression', 
                            num_leaves=5,
                            learning_rate=0.05,
                            n_estimators=50000,
                            max_bin=55,
                            bagging_fraction=0.8,
                            bagging_freq=5,
                            feature_fraction=0.2319,
                            feature_fraction_seed=9,
                            bagging_seed=9,
                            min_data_in_leaf=6,
                            min_sum_hessian_in_leaf=11)
lgb_pred, lgb_model = train_model(LGBoost, x_train, y_train, x_val)
print('LGBoost RMSE:', rmse(lgb_pred, y_val))
predict_test(xgb_model, 'lgb-pred.csv')

## Neural Network Approach

In [ ]:
NN_model = Sequential()

NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()


checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(x_train.values, y_train, epochs=500, batch_size=32, validation_split=0.2, callbacks=callbacks_list)

In [ ]:
y_pred = NN_model.predict(x_val.values)
print('NN RMSE: ', rmse(y_pred, y_val))

In [ ]:
weights_file = 'Weights-480--259.85519.hdf5'
NN_model.load_weights(weights_file)
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
y_pred = NN_model.predict(x_val.values)
print('Best Neural Network RMSE: ', rmse(y_pred, y_val))
predict_test(NN_model, 'neural-net-pred.csv', True)

In [ ]:
plt.scatter(y_val, y_pred)
plt.show()

## Stacked Regression models

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                        
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (GBoost, LGBoost),
                                                 meta_model = XGBoost)
stacked_pred, stacked_model = train_model(stacked_averaged_models, x_train.values, y_train.values, x_val.values)
print('Stacked model RMSE:', rmse(stacked_pred, y_val))
predict_test(stacked_model, 'stacked-pred.csv', True)